## Бейзлайн - простейший плоский классификатор на основе библиотеки FastText.

In [1]:
import os
from pathlib import Path
import fasttext
import numpy as np 
import pandas as pd
import csv
from gensim.utils import simple_preprocess

from HierarchicalLibrary import CategoryTree

Подготавливаем полный, тренировочный и валидационный датасеты:
перемешиваем данные в фрейме,
удаляем колонки рейтинга и кол-ва отзывов,
корректируем типы данных колонок,
заполняем пропущенные значения,
текст из колонок 'title', 'short_description' и 'name_value_characteristics' объединяем в колонку "Document", добавляем первые слова из колонки 'title', чтобы увеличить их вес (самые важные слова - в начале описания).

In [2]:
# Method for increasing the weight of the first words of title
def word_pyramid(string: str, min_n_words: int, max_n_words: int) -> list:
    result = []
    split = string.split(' ')
    for i in range(min_n_words, max_n_words+1):
        result += split[:i]
    return ' '.join(result)

# Predicting labels and probabilities for list of documents 
def predict_proba(documents: list) -> tuple:
    prediction = model.predict(documents, k=1)
    labels_result = []
    proba_result = []
    for label in prediction[0]:
        labels_result.append(int(label[0][9:]))
    return np.array(labels_result), np.array(prediction[1])[:, 0]

# Predicting on a single input
def predict(document):
    return int(model.predict(document)[0][0][9:])

# Test data preparation
def get_prepared_test_data(df_in: pd.DataFrame) -> pd.DataFrame:
    df=df_in.copy()
    df.drop(['rating', 'feedback_quantity'], axis=1, inplace=True)
    df.title = df.title.astype('string')
    df.short_description = df.short_description.astype('string')
    df.fillna(value='', inplace=True)
    df.name_value_characteristics = df.name_value_characteristics.astype('string')
    df = df.assign(Document=[str(x) + ' ' + str(y) + ' ' + str(z) + ' ' + word_pyramid(x, 2, 3) for x, y, z in zip(df['title'], df['short_description'], df['name_value_characteristics'])])
    df.drop(['title', 'short_description', 'name_value_characteristics'], axis=1, inplace=True)
    df.Document = df.Document.astype('string')
    df.Document = df.Document.apply(lambda x: ' '.join(simple_preprocess(x)))
    return df


In [3]:
full_train_data = pd.read_parquet('train.parquet')

data_full = full_train_data.sample(frac=1, random_state=1).copy()
data_full.drop(['rating', 'feedback_quantity'], axis=1, inplace=True)
data_full.title = data_full.title.astype('string')
data_full.short_description = data_full.short_description.astype('string')
data_full.fillna(value='', inplace=True)
data_full.name_value_characteristics = data_full.name_value_characteristics.astype('string')
data_full = data_full.assign(Document=[str(x) + ' ' + str(y) + ' ' + str(z) + ' ' + word_pyramid(x, 2, 3) for x, y, z in zip(data_full['title'], data_full['short_description'], data_full['name_value_characteristics'])])
data_full.drop(['title', 'short_description', 'name_value_characteristics'], axis=1, inplace=True)
data_full.Document = data_full.Document.astype('string')

data = data_full[:-4000].reset_index(drop=True)
data_valid = data_full[-4000:].reset_index(drop=True)


Преобразуем данные в формат, принимаемый FastText.

In [4]:
data.Document = data.Document.apply(lambda x: ' '.join(simple_preprocess(x)))
data_valid.Document = data_valid.Document.apply(lambda x: ' '.join(simple_preprocess(x)))

data.category_id = data.category_id.apply(lambda x: '__label__' + str(x))
data_valid.category_id = data_valid.category_id.apply(lambda x: '__label__' + str(x))

FastText принимает данные в виде текстовых файлов, поэтому сохраняем данные на диск.

In [5]:
# Saving the CSV file as a text file to train/test the classifier
data[['Document', 'category_id']].to_csv('train_fasttext.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

data_valid[['Document', 'category_id']].to_csv('test_fasttext.txt', 
                                               index = False, 
                                               sep = ' ',
                                               header = None, 
                                               quoting = csv.QUOTE_NONE, 
                                               quotechar = "", 
                                               escapechar = " ")


Обучаем модель.

In [6]:
# Training the fastText classifier
model = fasttext.train_supervised('train_fasttext.txt',
                                  lr=0.35,                # learning rate [0.1]
                                  dim=70,               # size of word vectors [100]
                                  ws=4,                # size of the context window [5]
                                  epoch=30,             # number of epochs [5]
                                  neg=5,               # number of negatives sampled [5]
                                  wordNgrams=3)

Read 4M words
Number of words:  87455
Number of labels: 1231
Progress: 100.0% words/sec/thread:   16580 lr:  0.000000 avg.loss:  0.276811 ETA:   0h 0m 0s 0.340789 avg.loss:  3.968186 ETA:   0h21m 7sm15s 1.434904 ETA:   0h24m 1s 20.0% words/sec/thread:   17664 lr:  0.280020 avg.loss:  1.007461 ETA:   0h22m26s 22.1% words/sec/thread:   17486 lr:  0.272728 avg.loss:  0.934059 ETA:   0h22m 4s 23.4% words/sec/thread:   17486 lr:  0.268059 avg.loss:  0.888414 ETA:   0h21m41s 52.2% words/sec/thread:   16891 lr:  0.167219 avg.loss:  0.472346 ETA:   0h14m 0s  16827 lr:  0.125070 avg.loss:  0.396615 ETA:   0h10m31s lr:  0.086577 avg.loss:  0.349536 ETA:   0h 7m18s 78.4% words/sec/thread:   16706 lr:  0.075487 avg.loss:  0.338099 ETA:   0h 6m23s 81.2% words/sec/thread:   16689 lr:  0.065664 avg.loss:  0.328538 ETA:   0h 5m34s avg.loss:  0.328457 ETA:   0h 5m33s 84.8% words/sec/thread:   16624 lr:  0.053073 avg.loss:  0.317162 ETA:   0h 4m31s lr:  0.043532 avg.loss:  0.309179 ETA:   0h 3m42s51s 98

Проверяем качество классификации на трейне:

In [7]:
# Evaluating performance on the entire train file
_, precision, recall = model.test('train_fasttext.txt') 
leaf_F1 = (2*precision*recall) / (precision+recall)

In [8]:
print(f'Leaf F1={leaf_F1:.4f}') 

Leaf F1=0.9862


Проверяем качество классификации на тестовой выборке:

In [9]:
# Evaluating performance on the entire test file
_, precision, recall = model.test('test_fasttext.txt')                      
leaf_F1 = (2*precision*recall) / (precision+recall)

In [10]:
print(f'Leaf F1={leaf_F1:.4f}') 

Leaf F1=0.8592


При необходимости, сохраняем или загружаем модель.

In [11]:
# Save the trained model
path = os.path.join(Path(".").parent, 'FastText_baseline', 'fasttext_model')
model.save_model(path)

In [12]:
# Load the trained model
path = os.path.join(Path(".").parent, 'FastText_baseline', 'fasttext_model')
model = fasttext.load_model(path)

Для того чтобы проверить качество иерархической классификации, инициализируем и заполняем класс дерева категорий.

In [13]:
cat_tree_df = pd.read_csv('categories_tree.csv', index_col=0)

In [14]:
cat_tree = CategoryTree()
cat_tree.add_nodes_from_df(cat_tree_df, parent_id_col='parent_id', title_col='title')
cat_tree.add_goods_from_df(data_full, category_id_col='category_id', good_id_col='id')

Считаем метрики для train датасета.

In [15]:
data_test = data[:10000].copy()
data_test.category_id = data_test.category_id.apply(lambda text: text[9:]).astype('int')
data_test['predicted_id'] = data_test.Document.astype('string')
data_test.predicted_id = data_test.predicted_id.apply(lambda text: predict(text)).astype('int')

test_target = data_test.category_id.tolist()
pred_leafs = data_test.predicted_id.tolist()

print(f'Train hF1={cat_tree.hF1_score(test_target, pred_leafs):.4f}') #0.9187
print(f'Train hF1_01={cat_tree.hF1_score_01(test_target, pred_leafs):.4f}') #0.9463

Train hF1=0.9920
Train hF1_01=0.9947


Предсказываем категории в тестовом сете.

In [16]:
data_valid_test = data_valid.copy()
data_valid_test.category_id = data_valid_test.category_id.apply(lambda text: text[9:]).astype('int')
data_valid_test['predicted_id'] = data_valid_test.Document.astype('string')
data_valid_test.predicted_id = data_valid_test.predicted_id.apply(lambda text: predict(text)).astype('int')

Подготавливаем данные для расчета иерархической метрики.

In [17]:
test_target = data_valid_test.category_id.tolist()
pred_leafs = data_valid_test.predicted_id.tolist()

Расчет иерархической F1-меры. 

In [18]:
print(f'Validation hF1={cat_tree.hF1_score(test_target, pred_leafs):.4f}') #0.9187
print(f'Validation hF1_01={cat_tree.hF1_score_01(test_target, pred_leafs):.4f}') #0.9463

Validation hF1=0.9157
Validation hF1_01=0.9443


Несмотря на простоту алгоритма, после подбора гиперпараметров получился очень хороший бейзлайн, hF1=0.91.

#### Обучаем модель на полном датасете

In [19]:
data_full.Document = data_full.Document.apply(lambda x: ' '.join(simple_preprocess(x)))
data_full.category_id = data_full.category_id.apply(lambda x: '__label__' + str(x))
data_full

,id,category_id,Document
143374,1181186,__label__12350,маска masil для объёма волос ml корейская косм...
194087,304936,__label__12917,силиконовый дорожный контейнер футляр чехол дл...
13188,816714,__label__14125,тканевая маска для лица муцином улитки snail д...
28368,1437391,__label__11574,браслеты из бисера браслеты из бисера браслеты...
181331,1234938,__label__12761,бальзам haute couture luxury blond для блондир...
...,...,...,...
21440,982751,__label__11567,цепь чокер женская цепь чокер женская цепь чок...
117583,747972,__label__12751,школьный бант школьный бант школьный бант школ...
73349,832637,__label__12454,наклейка для дизайна ногтей lucky rose тема ли...
267336,1378353,__label__11745,ключик замочек ключик ключик замочек


In [20]:
# Saving the CSV file as a text file to train/test the classifier
data_full[['Document', 'category_id']].to_csv('full_train_fasttext.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")


In [21]:
# Training the fastText classifier
model = fasttext.train_supervised('full_train_fasttext.txt',
                                  lr=0.35,                # learning rate [0.1]
                                  dim=70,               # size of word vectors [100]
                                  ws=4,                # size of the context window [5]
                                  epoch=30,             # number of epochs [5]
                                  neg=5,               # number of negatives sampled [5]
                                  wordNgrams=3)

Read 5M words
Number of words:  87958
Number of labels: 1231
Progress: 100.0% words/sec/thread:   16189 lr:  0.000000 avg.loss:  0.276396 ETA:   0h 0m 0s 14.1% words/sec/thread:   16159 lr:  0.300725 avg.loss:  1.249072 ETA:   0h26m42s  16142 lr:  0.281173 avg.loss:  0.987788 ETA:   0h25m 0s 0.275379 avg.loss:  0.923179 ETA:   0h24m10s  16372 lr:  0.274451 avg.loss:  0.914931 ETA:   0h24m 3s 23.8% words/sec/thread:   16342 lr:  0.266787 avg.loss:  0.853700 ETA:   0h23m26s 0.247369 avg.loss:  0.729115 ETA:   0h21m47s 40.5% words/sec/thread:   16020 lr:  0.208243 avg.loss:  0.570555 ETA:   0h18m39s lr:  0.205689 avg.loss:  0.562452 ETA:   0h18m25s 42.0% words/sec/thread:   16026 lr:  0.202888 avg.loss:  0.554145 ETA:   0h18m10s16m48s 0.434544 ETA:   0h13m13s  0h 9m15s avg.loss:  0.336106 ETA:   0h 6m25s 87.8% words/sec/thread:   16107 lr:  0.042557 avg.loss:  0.309344 ETA:   0h 3m47s


#### Подготовка данных для сабмита

Так как ничего значительно лучше бейзлайна обучить не удалось - будем использовать именно его.

In [23]:
# TEST data for submit
TEST_data = get_prepared_test_data(pd.read_parquet('test.parquet'))

In [24]:
# Test data for testing of submit data preparing functions
test_TEST_data = get_prepared_test_data(pd.read_parquet('train.parquet')[-10000:])
test_TEST_category_id = test_TEST_data.category_id.tolist()
test_TEST_data.drop(['category_id'], axis=1, inplace=True)

In [25]:
TEST_data

,id,Document
0,1070974,браслет из натуральных камней lotus браслет из...
1,450413,fusion life шампунь для сухих окрашенных волос...
2,126857,микрофон для пк jack мм всенаправленный универ...
3,1577569,серьги гвоздики сердце серьги гвоздики сердце ...
4,869328,чёрно красная стильная брошь тюльпаны из акрил...
...,...,...
70859,967535,носки мехом куницы авокадо разноцветные пуховы...
70860,1488636,эфирное масло сосны мл от кедрмаркет масло сос...
70861,827510,компект футболка шорты отличный комплект удобн...
70862,529244,купальный костюм mark formelle none российский...


In [26]:
def get_predicted_data(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    df['predicted_category_id'] = df.Document.astype('string')
    df.predicted_category_id = df.predicted_category_id.apply(lambda text: predict(text)).astype('int')
    df.drop(['Document'], axis=1, inplace=True)
    return df

Сохраняем окончательный сабмит:

In [27]:
get_predicted_data(TEST_data).to_parquet('result.parquet')